In [ ]:
import time
import torch

from diffusers import StableDiffusionPipeline, DDIMScheduler

In [ ]:
def save_and_display(image, pred_path):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    image.save(pred_path + "/" + timestr + ".png")
    display(image)

In [ ]:
model_path =  "/home/ubuntu/model"
predict_path = "/home/ubuntu/output"
!mkdir -p "$predict_path"

token_name = "aabbccddeeffgg"
class_str = "person"
feature_str = ""
num_pred_steps = 75
guide = 7.5
ddim = True

if ddim:
    ddim_scheduler = DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=False,
        )

    pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=ddim_scheduler,
        safety_checker=None).to("cuda")
else:
    pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        safety_checker=None).to("cuda")    

In [ ]:
prompts = [
        "colorful cinematic still of __token__ __class__ __feature__, armor, visible face, attractive person, cyberpunk,background made of brain cells, back light, organic, art by greg rutkowski, ultrarealistic, leica 30mm",
        "pencil sketch of  __token__ __class__ __feature__, inpired by greg rutkowski, digital art by artgem",
        "photo, colorful cinematic still of  __token__ __class__ __feature__, organic armor, cyberpunk, background brain cells mesh, art by greg rutkowski",
        "astronaut stylized beautiful painting digital artwork of __token__ __class__ __feature__, 4k, octane, trending on art station",
        "colorful cinematic still of __token__ __class__ __feature__, __token__ __class__ __feature__, with long hair, color lights, on stage, ultrarealistic",
        "photo, colorful cinematic still of  __token__ __class__ __feature__, with organic armor, cyberpunk background,  __token__ __class__ __feature__, greg rutkowski",
        "photo of  __token__ __class__ __feature__, astronaut, astronaut, face visible helmet in alien world abstract oil painting, greg rutkowski, detailed face",
        "photo of  __token__ __class__ __feature__, as firefighter, helmet, ultrarealistic, leica 30mm",
        "photo of  __token__ __class__ __feature__, as steampunk warrior, neon organic vines, digital painting",
        "impressionist painting of  __token__ __class__ __feature__, person by Daniel F Gerhartz, (( __token__ __class__ __feature__, with painted in an impressionist style)), nature, trees",
        "19th century oil portait of __token__ __class__ __feature__, ca. 1927, London National Gallery",
        "Highly detailed cyberpunk oil pastel drawing of __token__ __class__ __feature__, art by greg rutowski, ultrarealistic, leica 30mm",
        "20th century oil portrait painting of __token__ __class__ __feature__, as roman emperor wearing a wreath, highly detailed",
        "digital artwork detailed sketch of __token__ __class__ __feature__, face as a japanese ronin samurai trending on artstation",
        "19th century pastiche portrait of __token__ __class__ __feature__, as Emperor of France wearing royal regalia, ca. 1928 London National Gallery",
        "Hand drawn sketch of __token__ __class__ __feature__, portrait from leonardo davinci's The Codex Atlanticus is a 12-volume, ca. 1482, private collection",
        "Close up detailed portrait 19th century oil Biblical painting of __token__ __class__ __feature__, bringing the 10 commandment tablets down from mount sinai, oil painting",
        "Highly detailed modern oil painting 20st century of  __token__ __class__ __feature__, oil portrait by wassily kandinsky, Sotheby's",
        "Time's Person of the Year 1980, magazine cover portrait __token__ __class__ __feature__, co-founder and CEO",
        "48th President of the United States, __token__ __class__ __feature__, official portrait, The Smithsonian",
        "highly detailed photocopy of entire 1980 $100 Dollar Bill Currency engraving __token__ __class__ __feature__",
        "highly detailed photocopy of classified documents showing __token__ __class__ __feature__, of interest. CIA Museum, Langley Virginia."
]

In [ ]:
# Fixed the latent (you need to re-run this cell to choose a different seed.)
seed = 111
generator = torch.Generator(device="cuda")
generator = generator.manual_seed(seed)

width = 512
height = 512
latents = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    device = "cuda"
).half()

In [ ]:
# No additional feature string
feature_str = ""
fill_placeholders = lambda x: x.replace("__token__", token_name).replace("__class__", class_str).replace("__feature__", feature_str)
for prompt in prompts:
    prompt = fill_placeholders(prompt)
    image = pipe(prompt, num_inference_steps=num_pred_steps, guidance_scale=guide, latents = latents).images[0]
    print(prompt)
    save_and_display(image, predict_path)